In [ ]:
from typing import Literal, Callable, Any
import math, time, json, random
from random import randint
from collections import defaultdict

from drivingppo.common import set_seed, SPD_SCFAC, DIS_SCFAC, WORLD_DT, ACTION_REPEAT, LOOKAHEAD_POINTS
from drivingppo.environment import speed_norm
from drivingppo.world import World, Car, create_empty_map, angle_of, distance_of, pi, pi2, rad_to_deg

from world_samples import NEAR, W_CONFIG, gen_1, gen_2, gen_3, gen_3l, generate_random_world_plain, gen_obs

import numpy as np
import pandas as pd


def get_pp_lookahead_point(world:World, ld:float):
    p = world.player
    px, pz = p.x, p.z
    waypoints = world.waypoints
    idx = world.waypoint_idx

    lookahead_point = world.get_curr_wpoint()
    found = False

    # 현재 위치부터 남은 웨이포인트들을 선분으로 이어 교차점 검사
    pts = [(px, pz)] + waypoints[idx:]

    i = 0  # 반환되는 lookahead_point 다음의 웨이포인트는 몇 번째? (현재목표점은 0) pts의 0번 요소가 에이전트이므로 0번 wp는 pts의 1번 요소임.
    for i in range(len(pts) - 1):
        wp1x, wp1z = pts[i]
        wp2x, wp2z = pts[i+1]

        # 선분(wp1-wp2)과 원(중심:(px,pz), 반지름:ld)의 수학적 교차점 계산
        fx, fz = wp1x - px,   wp1z - pz
        dx, dz = wp2x - wp1x, wp2z - wp1z

        # f:p~wp1 | d:wp1~wp2 | (f*t + d)^2 = l^2 | t에대한 2차방정식 풀이
        a = dx*dx + dz*dz
        b = 2 * (fx*dx + fz*dz)
        c = (fx*fx + fz*fz) - ld*ld

        D = b*b - 4*a*c  # discriminant
        if D >= 0 and a > 0.00001:
            D = math.sqrt(D)
            t1 = (-b - D) / (2*a)
            t2 = (-b + D) / (2*a)

            # 진행 방향에 있는 선분 위의 교차점 (0 <= t <= 1) 선택
            if 0 <= t2 <= 1:  # t2가 더 앞서 있는 점이므로 우선 채택
                lookahead_point = (wp1x + t2*dx, wp1z + t2*dz)
                found = True
                break
            elif 0 <= t1 <= 1:
                lookahead_point = (wp1x + t1*dx, wp1z + t1*dz)
                found = True
                break

    if found:
        return lookahead_point, i

    # 교차점을 못 찾았다면 (목표가 너무 가깝거나 경로 끝인 경우) 다음 웨이포인트를 쳐다봄
    return world.get_curr_wpoint(0), 0


class PurePursuitController:
    def __init__(self,
                 target_speed=13.0,
                 min_ld=1.4,
                 ld_k=1.0,
                 kp_steer=1.9,
                 k_cornering=0.15,
                 k_accel=0.4,
    ):
        self.target_speed = target_speed # 목표 주행 속도 (m/s)
        self.min_ld       = min_ld       # 전방 주시 거리 최소 (m)
        self.ld_k         = ld_k         # 속도 비례 전방 주시 계수 (Ld = min_ld + ld_k * speed)
        self.kp_steer     = kp_steer     # 조향 비례 제어 게인
        self.k_cornering  = k_cornering  # 선회시 감속 정도
        self.k_accel      = k_accel      # 종방향 가속 게인

    def get_action(self, world:World) -> tuple[float, float, bool]:
        """
        World 개체를 받아 액션(ws, ad, stop) 반환
        """
        p = world.player
        px, pz = p.x, p.z

        # 이미 도착한 경우 정지
        if world.arrived:
            return 0.0, 0.0, True

        ld = self.min_ld + self.ld_k * max(0, p.speed)
        lookahead_point, _ = get_pp_lookahead_point(world, ld)

        # 조향 제어 (ad)
        tx, tz = lookahead_point
        abs_angle = angle_of(px, pz, tx, tz)

        rel_angle = abs_angle - p.angle_x
        rel_angle = (rel_angle + math.pi) % (2*math.pi) - math.pi  # 정규화: -pi ~ pi

        ad = self.kp_steer * rel_angle
        # 종방향 제어 (ws)
        current_target_speed = self.target_speed * (1.0 - ad*ad * self.k_cornering)
        ad = max(min(ad, 1.0), -1.0)  # 액션 유효범위로 클리핑

        speed_error = current_target_speed - p.speed
        ws = speed_error * 0.5  # 가속도 비례 게인
        ws = max(min(ws, 1.0), -1.0)

        return ws, ad, False


def evaluate(
        con:PurePursuitController,
        world_generator: Callable[[], World],
        episode_num=100,
        verbose: bool=True,
        csv_path: str="",
        time_step=WORLD_DT,
        action_repeat=ACTION_REPEAT,
        max_esteps=2000  # 무한 루프 방지용 최대 스텝 수
) -> dict:

    print(f"Pure Pursuit {episode_num}회 에피소드 평가 시작...")

    all_metrics = defaultdict(list)
    episode_lengths = []
    success_count = 0

    for i in range(episode_num):
        i1 = i + 1
        checkPeriod = max(1, episode_num // 10)

        world = world_generator()

        estep_count = 0
        ws_penalty_sum = 0.0
        ad_penalty_sum = 0.0
        speed_list = []
        action_history = []

        done = False

        # 에피소드 진행
        while not done and estep_count < max_esteps:
            estep_count += 1

            ws, ad, _ = con.get_action(world)
            action_history.append([ws, ad])

            # --- 물리 스텝 실행 ---
            for _ in range(action_repeat):
                world.set_action(ws, ad)
                world.step(time_step)

            # --- 지표 수집 ---
            s_norm = speed_norm(world.player.speed)
            ws_penalty = (- ws * s_norm * 4.0) if (ws * s_norm > 0) else 0.0

            ws_penalty_sum += ws_penalty
            ad_penalty_sum += (- ad * ad * 1.7)
            speed_list.append(world.player.speed)

            # --- 종료 조건 검사 (도달, 경로 이탈, 충돌 등) ---
            if world.arrived or getattr(world, 'lost', False) or getattr(world, 'player_collision', False):
                done = True

        # 에피소드 단위 지표 집계
        tcount = estep_count * action_repeat * time_step / 1000.0
        final_ws = ws_penalty_sum / tcount if tcount > 0 else 0.0
        final_ad = ad_penalty_sum / tcount if tcount > 0 else 0.0
        speed_mean = np.mean(speed_list) if speed_list else 0.0
        speed_var  = np.var(speed_list) if speed_list else 0.0
        if len(action_history) > 0:
            action_arr = np.array(action_history)
            ws_diff_mean = float(np.mean(np.abs(np.diff(action_arr[:, 0]))))  if len(action_history) > 1  else 0.0
            ad_sq_mean   = float(np.mean(np.square(action_arr[:, 1])))
        else:
            ws_diff_mean, ad_sq_mean = 0.0, 0.0

        is_success = world.arrived
        if is_success: success_count += 1

        episode_lengths.append(estep_count)

        all_metrics['ending/estep'].append(estep_count)
        all_metrics['current_time'].append(tcount)
        all_metrics['rewards/6.ws'].append(final_ws)
        all_metrics['rewards/7.ad'].append(final_ad)
        all_metrics['metrics/speed_mean'].append(speed_mean)
        all_metrics['metrics/speed_var'].append(speed_var)
        all_metrics['metrics/ws_diff_mean'].append(ws_diff_mean)
        all_metrics['metrics/ad_sq_mean'].append(ad_sq_mean)
        all_metrics['success'].append("True" if is_success else "False") # 범주형 출력을 위해 문자열 처리

        # 진행 상황 출력
        if i1 % checkPeriod == 0:
            if verbose:
                status = "성공" if is_success else "실패/이탈"
                print(f"[{i1}/{episode_num}] 완료 - 상태: {status}, Steps: {estep_count}, Avg Speed: {speed_mean:.2f}")

    # 평가 결과 출력 및 저장
    if verbose:
        print("\n" + "="*41)
        print(f"평가 결과 ({episode_num} 에피소드 평균)")
        print("="*41)
        print(f"Success Rate  : {(success_count / episode_num) * 100:.1f}% ({success_count}/{episode_num})")
        print(f"Episode Length: {np.mean(episode_lengths):.1f} ± {np.std(episode_lengths):.1f}")

    if all_metrics:
        df_metrics = pd.DataFrame(all_metrics)

        # 수치형: 평균, 표준편차
        num_df = df_metrics.select_dtypes(include=[np.number])
        if not num_df.empty and verbose:
            print("-" * 41)
            summary = num_df.describe().loc[['mean', 'std']].T
            summary['mean'] = summary['mean'].map('{:.4f}'.format)  # 지수표기 안 함.
            print(summary)

        # 문자열(범주형): 종류별 비율
        cat_df = df_metrics.select_dtypes(exclude=[np.number])
        if not cat_df.empty and verbose:
            print("-" * 41)
            for col in cat_df.columns:
                print(f"\n* {col}")
                counts = df_metrics[col].value_counts()
                ratios = df_metrics[col].value_counts(normalize=True)

                for idx, val in counts.items():
                    ratio = ratios[idx] * 100  #type:ignore
                    print(f"   - {idx:<10}: {val:3d}회 ({ratio:5.1f}%)")

        # CSV 저장
        if csv_path:
            df_metrics.to_csv(csv_path, index=False)
            if verbose:
                print(f"\n💾 세부 결과 저장됨: {csv_path}")

    return all_metrics

In [ ]:
from drivingppo.simsim import WorldViewer

def testew():
    obstacle_map = create_empty_map(100, 100)
    w = World(
        wh=(100, 100),
        player=Car({
            'playerPos': {'x': 10, 'z': 10},
            'playerBodyX': 0.0,
            'playerSpeed': 0.,
        }),
        obstacle_map=obstacle_map,
        waypoints=[
            (10, 50),
            (20, 50),
            (50, 70),
            (60, 60),
        ],
        config=W_CONFIG|{
            'map_border': False,
            'far': 999.9
        }
    )
    return w


def run(
        world_generator:Callable[[], World],
        con:PurePursuitController,
        time_spd=2.0,
        time_step=WORLD_DT,
        action_repeat=ACTION_REPEAT,
        auto_close_at_end=True,
    ):

    world = world_generator()
    viewer = WorldViewer(world, auto_update=False)

    # 지표 수집용 변수 초기화
    estep_count = 0
    ws_penalty_sum = 0.0
    ad_penalty_sum = 0.0
    speed_list = []

    while not world.arrived:

        estep_count += 1
        ws, ad, _ = con.get_action(world)  # 컨트롤러가 조작 산출

        s_norm = speed_norm(world.player.speed)
        ws_penalty = (- ws * s_norm * 4.0) if (ws * s_norm > 0) else 0.0
        ws_penalty_sum += ws_penalty
        ad_penalty_sum += (- ad * ad * 1.7)
        speed_list.append(world.player.speed)

        for _ in range(action_repeat):
            world.set_action(ws, ad)  # 행동 실행
            world.step(time_step)

        if viewer.closed: break
        viewer.update()  # 시각화 호출
        time.sleep(time_step / 1000.0 / time_spd)  # 시각화 프레임을 위해 딜레이 추가

    tcount = estep_count * action_repeat * time_step / 1000.0  # 흐른 시간 (초)
    final_ws = ws_penalty_sum / tcount
    final_ad = ad_penalty_sum / tcount
    speed_mean = np.mean(speed_list)  if speed_list else 0.0
    speed_var  = np.var(speed_list)   if speed_list else 0.0

    print("\n=== Pure Pursuit 에피소드 종료 지표 ===")
    print(f'current_time: {tcount}')
    print(f'ending/estep: {estep_count}')
    print(f"rewards/6.ws      : {final_ws:.4f}")
    print(f"rewards/7.ad      : {final_ad:.4f}")
    print(f"metrics/speed_mean: {speed_mean:.4f}")
    print(f"metrics/speed_var : {speed_var:.4f}")
    print("=======================================\n")

    if auto_close_at_end:
        viewer.close()
    else:
        viewer.occupy_mainloop()

def gen_3l(): return generate_random_world_plain(map_h=300, map_w=300, num=10,               min_dist=NEAR*2, max_dist=DIS_SCFAC, ang_init='rand', ang_lim=pi*1.0, spd_init='rand')  # 테스트용: 같은 패턴인데 좀 긴 경로

con = PurePursuitController()
set_seed(7)
run(gen_3l, con, auto_close_at_end=False)

In [ ]:
set_seed(0)
metrics = evaluate(
    con=PurePursuitController(),
    world_generator=gen_3l,
    episode_num=100,
    verbose=True,
    csv_path="pp.csv"
)

In [ ]:
from typing import Callable
import os
import itertools
import numpy as np
import pandas as pd

# 기존 모듈 임포트 유지 (환경에 맞게 조정 필요)
from drivingppo.common import set_seed, WORLD_DT, ACTION_REPEAT
from drivingppo.environment import speed_norm
from drivingppo.world import World
from world_samples import NEAR, gen_1, gen_2, gen_3, gen_3l, generate_random_world_plain

FILE_PATH = "pp_grid_search_results.csv"

param_grid = {
    'target_speed': [10.0, 11.0, 12.0, 13.0, 14.0],
    'min_ld':       [1.0, 1.2, 1.4, 1.6, 1.8, 2.0],
    'ld_k':         [0.4, 0.6, 0.8, 1.0, 1.2, 1.4],
    'kp_steer':     [1.4, 1.5, 1.6, 1.7, 1.8, 1.9],
    'k_cornering':  [0.15, 0.2, 0.25, 0.3],
    'k_accel':      [0.3, 0.4, 0.5, 0.6, 0.7, 0.8],
}

def _evaluate_pp_fast__process_world(
        con:PurePursuitController,
        world:World,
        max_esteps:int,
) -> float|None:
    estep_count = 0

    while not world.arrived:
        # 에피소드 진행
        estep_count += 1

        # world 진행
        ws, ad, _ = con.get_action(world)
        for _ in range(ACTION_REPEAT):
            world.set_action(ws, ad)
            world.step(WORLD_DT)

        # 1차 조기탈락 판정: 한 번이라도 완주 실패이면 탈락
        if world.lost \
        or estep_count >= max_esteps:
            return None

    return estep_count

def evaluate_pp_fast(
        con:PurePursuitController,
        world_generator:Callable[[], World],
        episode_num=100,
        midcheck_num=10,
        max_esteps=250,
        early_stop_mean=150.0,
) -> float|None:
    '''
    소요시간만으로 평가

    Returns: 평균 소요 estep 수; None: 탈락
    '''
    if midcheck_num >= episode_num:
        raise ValueError(f'중간확인점({midcheck_num}) > 전체({episode_num})')

    episode_lengths = []

    estep_sum = 0

    for _ in range(midcheck_num):
        estep_count = _evaluate_pp_fast__process_world(con, world_generator(), max_esteps)
        if estep_count is None:
            return None

        # 결과 누적
        episode_lengths.append(estep_count)
        estep_sum += estep_count

    # 2차 조기탈락 판정: 일단 조금만 해보고 성적 안 좋으면 탈락.
    estep_mean = estep_sum / midcheck_num
    if estep_mean > early_stop_mean:
        return None

    for _ in range(midcheck_num, episode_num):
        estep_count = _evaluate_pp_fast__process_world(con, world_generator(), max_esteps)
        if estep_count is None:
            return None

        # 결과 누적
        episode_lengths.append(estep_count)
        estep_sum += estep_count

    return estep_sum / episode_num


def run_grid_search_itertools(world_generator: Callable[[], World], output_path:str=FILE_PATH, delete_prev=True, seed=0):
    if delete_prev and os.path.exists(output_path):
        os.remove(output_path)

    keys   = param_grid.keys()
    values = param_grid.values()
    combinations = list(itertools.product(*values))

    total_combinations = len(combinations)
    print(f"파라미터 총 {total_combinations}개 조합 탐색 시작")

    for current_iter, combo in enumerate(combinations, 1):
        params = dict(zip(keys, combo))
        print(f"[{current_iter}/{total_combinations}] 평가 중: {params}", end="")

        con = PurePursuitController(**params)
        set_seed(seed, verbose=False)
        estep_mean = evaluate_pp_fast(con, world_generator, episode_num=100)

        if estep_mean is None:
            print(" -> ❌ 탈락")
            continue

        print(f" -> ✅ ({estep_mean})")

        row_data = params.copy()
        row_data['estep_mean'] = estep_mean

        df_iter = pd.DataFrame([row_data])
        write_header = not os.path.exists(output_path)
        df_iter.to_csv(output_path, mode='a', header=write_header, index=False)

    print(f"\n탐색 완료. '{output_path}'")


def analyze_grid_search_results(file_path=FILE_PATH, verbose=True):
    df = pd.read_csv(file_path)

    if df.empty:
        print("저장된 유효한 조합이 없습니다.")
        return df

    sorted_df = df.sort_values(
        by=['estep_mean'], # 소요 스텝 수 오름차순
        ascending=[True]
    )

    if verbose:
        print(sorted_df.to_string(index=False))

    return sorted_df

# 실행 예시
# run_grid_search_itertools(gen_3l, seed=100)
summary = analyze_grid_search_results(verbose=False)
summary.head(10)